In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
from pathlib import Path
import cv2
import torch
import gc

In [ ]:
rf = Roboflow(api_key="0az8ziNbTVpKHYaILb91")
project = rf.workspace("nsu-sxhmy").project("project_x-ar6bm")
version = project.version(7)
dataset = version.download("yolov11")

In [ ]:
# model = YOLO("yolo11n.pt")
# model = YOLO("yolo11s.pt")
model = YOLO("yolo11m.pt")

model.train(
    data=dataset.location + "/data.yaml",
    epochs=1000,
    patience=50,
    imgsz=640,
    batch=16,
    device="cuda",
    lr0=0.01,         # стартовый lr
    lrf=0.01,         # final lr fraction
    momentum=0.937,   # SGD momentum/Adam beta1
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,          # box loss gain
    cls=0.5,          # cls loss gain

    # # Light augs
    # hsv_h=0.0075,      # аугментация: hue
    # hsv_s=0.09,        # saturation
    # hsv_v=0.08,        # value
    # degrees=30.0,      # поворот
    # translate=0.01,    # сдвиг
    # scale=0.025,        # масштаб
    # shear=0.001,        # сдвиг (shear)
    # perspective=0.0001,  # перспектива
    # flipud=0.5,       # переворот вверх-вниз
    # fliplr=0.5,       # переворот влево-вправо
    # mosaic=0.25,       # mosaic аугментация
    # mixup=0.1,        # mixup аугментация
    # copy_paste=0.25,   # copy-paste аугментация

    # hsv_h=0.075,      # аугментация: hue
    # hsv_s=0.9,        # saturation
    # hsv_v=0.8,        # value
    # degrees=90.0,      # поворот
    # translate=0.1,    # сдвиг
    # scale=0.25,        # масштаб
    # shear=0.01,        # сдвиг (shear)
    # perspective=0.0001,  # перспектива
    # flipud=0.5,       # переворот вверх-вниз
    # fliplr=0.5,       # переворот влево-вправо
    # mosaic=0.25,       # mosaic аугментация
    # mixup=0.1,        # mixup аугментация
    # copy_paste=0.25,   # copy-paste аугментация

    # Chilobasik
    hsv_h=0.05,
    hsv_s=0.7,
    hsv_v=0.6,
    degrees=20.0,
    translate=0.1,
    scale=0.2,
    shear=0.01,
    perspective=0.001,
    flipud=0.2,
    fliplr=0.5,
    mosaic=0.5,
    mixup=0.1,
    copy_paste=0.25,

    save=True,
    save_period=5,
    val=True,
    plots=True,
    workers=8
)

In [ ]:
# model = YOLO(r"C:\Users\kuzga\OneDrive\Рабочий стол\CV_case_tbanc\runs\detect\train5\weights\best.pt")
# model = YOLO(r"C:\Users\kuzga\OneDrive\Рабочий стол\CV_case_tbanc\runs\detect\train11\weights\best.pt")
# model = YOLO(r"C:\Users\kuzga\OneDrive\Рабочий стол\CV_case_tbanc\runs\detect\train13\weights\best.pt")
# model = YOLO(r"C:\Users\kuzga\OneDrive\Рабочий стол\CV_case_tbanc\runs\detect\train14\weights\best.ptc:\Users\kuzga\OneDrive\")
# model = YOLO(r"")
# model = YOLO(r"")
# model = YOLO(r"")

In [ ]:
image_folder = Path("./data_sirius")
all_images = list(image_folder.glob("*.jpg"))

save_root = Path("./predicted_images")
original_dir = save_root / "original"
predicted_dir = save_root / "predicted"
original_dir.mkdir(parents=True, exist_ok=True)
predicted_dir.mkdir(parents=True, exist_ok=True)

# Параметры
batch_size = 100        # размер батча для predict
max_images = 5000     # максимум картинок всего
max_with_boxes = 200  # максимум картинок с боксами

images_processed = 0
images_with_boxes = 0

for i in range(0, min(len(all_images), max_images), batch_size):
    batch = all_images[i:i + batch_size]
    batch = [str(p) for p in batch]

    results = model.predict(
        source=batch,
        imgsz=640,
        conf=0.4,
        iou=0.1,
        save=False,
        save_txt=False
    )

    for r in results:
        images_processed += 1
        if len(r.boxes) > 0:
            # top_conf_threshold = 0.4
            # min_conf = float('inf')
            # max_conf = float('-inf')
            # for box in r.boxes:
            #     min_conf = min(min_conf, box.conf)
            #     max_conf = max(max_conf, box.conf)

            # if min_conf > top_conf_threshold:
            #     continue

            # if max_conf > top_conf_threshold:
            #     continue

            images_with_boxes += 1

            orig_path = original_dir / Path(r.path).name
            pred_path = predicted_dir / Path(r.path).name

            # Сохраняем оригинал
            cv2.imwrite(str(orig_path), r.orig_img)

            # Сохраняем с бокcами
            annotated = r.plot()
            cv2.imwrite(str(pred_path), annotated)

            del annotated

        # Проверяем лимит картинок с боксами
        if images_with_boxes >= max_with_boxes:
            break

    # --- очистка памяти ---
    del results
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Проверка лимита
    if images_with_boxes >= max_with_boxes:
        print(f"Достигнут лимит картинок с боксами: {max_with_boxes}")
        break

    print(f"Обработано картинок всего: {images_processed}, с боксами: {images_with_boxes}, доля: {images_with_boxes / images_processed}")

print(f"Обработано картинок всего: {images_processed}, с боксами: {images_with_boxes}, доля: {images_with_boxes / images_processed}")

In [ ]:
5:  1ая версия на 143 картинки, из них 20 негативных
11: 2ая версия на 243 картинки, из них 90 негативных
13: 3ая версия на 351 картинки, из них 180 негативных
17: 4ая версия на 612 картинки, из них > 300? негативных
18: 4ая версия на 612 картинки, из них > 300? негативных, лайтовые ауги
__: 5ая версия на 1700 картинки, из них > 500? негативных, по 2 и зкластера в валиде, по 5 из кластера в трейне, подспиздил разметку с робофлоу.